In [1]:
import networkx as nx
import os
import pandas as pd
import numpy as np
from collections import Counter
import datetime
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
processed_network = os.path.join(os.getenv("DATA_DIR"),"processed_network")

In [3]:
processed_network

'/Users/felisialoukou/Documents/govuk-network-embedding/data/processed_network'

In [4]:
nodefile = os.path.join(processed_network, "taxon_28_31_01_sample30_cs_nodes.csv.gz")
edgefile = os.path.join(processed_network, "taxon_28_31_01_sample30_cs_edges.csv.gz")

### Load nodes

In [5]:
nodes = pd.read_csv(nodefile, compression="gzip", sep="\t")

In [6]:
# nodes.reset_index(inplace=True)
# nodes.drop("Node_id",axis=1,inplace=True)
# nodes.rename(columns={'index':"Node_id"},inplace=True)

In [7]:
nodes.head()

,Node,Node_id,Node_Taxon
0,/government/publications/guidance-for-dependan...,0,"('d612c61e-22f4-4922-8bb2-b04b9202126e',)"
1,/visa-fees,1,"('29480b00-dc4d-49a0-b48c-25dda8569325',)"
2,/find-a-visa-application-centre,2,"('d3df7728-db52-411f-afd2-44dbd09f553b',)"
3,/entering-staying-uk/family-visas,3,"('d612c61e-22f4-4922-8bb2-b04b9202126e',)"
4,/uk-family-visa,4,"('d612c61e-22f4-4922-8bb2-b04b9202126e',)"


In [8]:
nodes.shape

(745581, 3)

In [9]:
nodes = nodes[~nodes.Node.str.contains(" ")]

### Load edges

In [10]:
edges = pd.read_csv(edgefile, compression="gzip", sep="\t")

In [11]:
# nid_dict = dict(zip(nodes.Node,nodes.Node_id))
# edges['Source_id'] = edges['Source_node'].map(nid_dict)
# edges['Destination_id'] = edges['Destination_node'].map(nid_dict)

In [12]:
edges.head()

,Source_node,Source_id,Destination_node,Destination_id,Weight,Source_Taxon,Destination_Taxon
0,/government/publications/guidance-for-dependan...,0,/visa-fees,1,24,"('d612c61e-22f4-4922-8bb2-b04b9202126e',)","('29480b00-dc4d-49a0-b48c-25dda8569325',)"
1,/visa-fees,1,/find-a-visa-application-centre,2,433,"('29480b00-dc4d-49a0-b48c-25dda8569325',)","('d3df7728-db52-411f-afd2-44dbd09f553b',)"
2,/find-a-visa-application-centre,2,/entering-staying-uk/family-visas,3,1,"('d3df7728-db52-411f-afd2-44dbd09f553b',)","('d612c61e-22f4-4922-8bb2-b04b9202126e',)"
3,/entering-staying-uk/family-visas,3,/uk-family-visa,4,148,"('d612c61e-22f4-4922-8bb2-b04b9202126e',)","('d612c61e-22f4-4922-8bb2-b04b9202126e',)"
4,/uk-family-visa,4,/uk-family-visa/partner-spouse,5,10863,"('d612c61e-22f4-4922-8bb2-b04b9202126e',)","('d612c61e-22f4-4922-8bb2-b04b9202126e',)"


#### Filter edge list: remove self-loops, remove edges to/from `/search`

In [22]:
edges[edges.Destination_node.str.contains("/search?")]

,Source_node,Source_id,Destination_node,Destination_id,Weight,Source_Taxon,Destination_Taxon
738,/get-information-about-property-and-land,488,/get-information-about-property-and-land/searc...,489,1851,"('55f1135f-2b81-440f-9d5b-52f593f4b9ad', '0c46...","('55f1135f-2b81-440f-9d5b-52f593f4b9ad', '0c46..."
741,/government/publications/inspection-and-applic...,490,/get-information-about-property-and-land/searc...,489,1,"('495afdb6-47be-4df1-8b38-91c8adb1eefc', '4ad6...","('55f1135f-2b81-440f-9d5b-52f593f4b9ad', '0c46..."
747,/,13,/search-property-information-land-registry,492,394,"('other',)","('55f1135f-2b81-440f-9d5b-52f593f4b9ad', '0c46..."
1312,/topic/intellectual-property/trade-marks,717,/search-trade-mark-decisions,725,299,"('d949275c-88f8-4623-a44b-eb3706651e10',)","('90c7fbe9-e78e-4b36-b3ba-c1637c28326d', '584e..."
2842,/government/organisations/land-registry,484,/topic/land-registration/searches-fees-forms,1496,5428,"('other',)","('other',)"
9496,/limited-company-formation/choose-company-name,170,/search-for-trademark,4258,594,"('386180ca-4219-478e-8b80-a271dd5ce5f2', 'c3bc...","('90c7fbe9-e78e-4b36-b3ba-c1637c28326d', '584e..."
11832,/government/statistics?keywords=&taxons[]=all&...,5214,/search-house-prices,5215,1,"('other',)","('55f1135f-2b81-440f-9d5b-52f593f4b9ad', '0c46..."
13869,/limited-company-formation/appoint-directors-a...,2615,/search-bankruptcy-insolvency-register,5222,34,"('386180ca-4219-478e-8b80-a271dd5ce5f2', 'c3bc...","('7c4cf197-2dba-4a82-83e2-6c8bb332525c',)"
17160,/wills-probate-inheritance/if-youre-an-executor,1159,/wills-probate-inheritance/searching-for-proba...,7297,57,"('3bc4ec93-fd86-4c66-98d0-7623cbbaa6be', '0fff...","('3bc4ec93-fd86-4c66-98d0-7623cbbaa6be', '0fff..."
17164,/wills-probate-inheritance/once-the-grants-bee...,7299,/wills-probate-inheritance/searching-for-proba...,7297,105,"('3bc4ec93-fd86-4c66-98d0-7623cbbaa6be', '0fff...","('3bc4ec93-fd86-4c66-98d0-7623cbbaa6be', '0fff..."


In [13]:
for value in edges[edges.Destination_node.str.contains("/search?")].head(10).values:
    print(value)
    print("=======")

['/get-information-about-property-and-land' 488
 '/get-information-about-property-and-land/search-the-register' 489 1851
 "('55f1135f-2b81-440f-9d5b-52f593f4b9ad', '0c46aba4-1986-4574-b957-734c6d104546')"
 "('55f1135f-2b81-440f-9d5b-52f593f4b9ad', '0c46aba4-1986-4574-b957-734c6d104546')"]
['/government/publications/inspection-and-application-for-official-copies'
 490 '/get-information-about-property-and-land/search-the-register' 489 1
 "('495afdb6-47be-4df1-8b38-91c8adb1eefc', '4ad66168-274d-4d83-9195-f4ab0ccc97cb', '82730dd2-4ead-4ae6-943b-2c910e758279')"
 "('55f1135f-2b81-440f-9d5b-52f593f4b9ad', '0c46aba4-1986-4574-b957-734c6d104546')"]
['/' 13 '/search-property-information-land-registry' 492 394 "('other',)"
 "('55f1135f-2b81-440f-9d5b-52f593f4b9ad', '0c46aba4-1986-4574-b957-734c6d104546')"]
['/topic/intellectual-property/trade-marks' 717
 '/search-trade-mark-decisions' 725 299
 "('d949275c-88f8-4623-a44b-eb3706651e10',)"
 "('90c7fbe9-e78e-4b36-b3ba-c1637c28326d', '584eeb89-a08e-4a

In [14]:
# import re
# re.match("+")
# for page in ["/search","/search/advanced/",
#              "/search-skf=1dks",
#              "/get-information-about-property-and-land/search-the-register",
#              "/search?w", "/search?q=form4+8"]:
#     if not (re.match(r"^/search[//?|/]", page) or page=="/search"):
#         print(page)

In [15]:
edges[edges.Destination_node.str.contains("/search?")].shape, edges[edges.Destination_node.str.
                                                                    contains("/search?")].Weight.sum()

((4032, 7), 229313)

In [16]:
edges = edges[(~edges.Source_node.str.contains(" ")) & (~edges.Destination_node.str.contains(" "))]
edges = edges[edges.Source_node!=edges.Destination_node]
# edges = edges[(~edges.Source_node.str.contains("/search?")) & (~edges.Destination_node.str.contains("/search?"))]

In [17]:
edges.shape

(2383883, 7)

In [18]:
edges[(edges.Source_node.str.contains("/search?"))]

,Source_node,Source_id,Destination_node,Destination_id,Weight,Source_Taxon,Destination_Taxon
739,/get-information-about-property-and-land/searc...,489,/get-information-about-property-and-land/copie...,485,1123,"('55f1135f-2b81-440f-9d5b-52f593f4b9ad', '0c46...","('55f1135f-2b81-440f-9d5b-52f593f4b9ad', '0c46..."
742,/get-information-about-property-and-land/searc...,489,/get-information-about-property-and-land,488,476,"('55f1135f-2b81-440f-9d5b-52f593f4b9ad', '0c46...","('55f1135f-2b81-440f-9d5b-52f593f4b9ad', '0c46..."
2843,/topic/land-registration/searches-fees-forms,1496,/government/collections/hm-land-registry-forms,1497,1324,"('other',)","('495afdb6-47be-4df1-8b38-91c8adb1eefc', '4ad6..."
9497,/search-for-trademark,4258,/limited-company-formation,169,20,"('90c7fbe9-e78e-4b36-b3ba-c1637c28326d', '584e...","('386180ca-4219-478e-8b80-a271dd5ce5f2', 'c3bc..."
11833,/search-house-prices,5215,/check-house-price-trends,5216,829,"('55f1135f-2b81-440f-9d5b-52f593f4b9ad', '0c46...","('other',)"
11846,/search-bankruptcy-insolvency-register,5222,/bankruptcy,5223,81,"('7c4cf197-2dba-4a82-83e2-6c8bb332525c',)","('7c4cf197-2dba-4a82-83e2-6c8bb332525c',)"
12244,/topic/land-registration/searches-fees-forms,1496,/government/collections/fees-hm-land-registry-...,4624,565,"('other',)","('495afdb6-47be-4df1-8b38-91c8adb1eefc', '4ad6..."
13870,/search-bankruptcy-insolvency-register,5222,/limited-company-formation/appoint-directors-a...,2615,20,"('7c4cf197-2dba-4a82-83e2-6c8bb332525c',)","('386180ca-4219-478e-8b80-a271dd5ce5f2', 'c3bc..."
17161,/wills-probate-inheritance/searching-for-proba...,7297,/wills-probate-inheritance/stopping-a-grant-of...,7298,149,"('3bc4ec93-fd86-4c66-98d0-7623cbbaa6be', '0fff...","('3bc4ec93-fd86-4c66-98d0-7623cbbaa6be', '0fff..."
17165,/wills-probate-inheritance/searching-for-proba...,7297,/search-will-probate,7300,15426,"('3bc4ec93-fd86-4c66-98d0-7623cbbaa6be', '0fff...","('0fffa994-b76d-4539-8bf9-2a6c6751580d',)"


In [19]:
edges.sort_values(by="Weight", ascending=False)[["Source_node","Destination_node","Weight"]].head()

,Source_node,Destination_node,Weight
252,/government/organisations/companies-house,/get-information-about-a-company,6977581
20,/log-in-file-self-assessment-tax-return,/log-in-file-self-assessment-tax-return/sign-i...,3627688
210,/government/organisations/hm-revenue-customs,/log-in-register-hmrc-online-services,3020415
226,/self-assessment-tax-returns,/log-in-file-self-assessment-tax-return,1936111
2299,/universal-credit,/sign-in-universal-credit,1804534


In [20]:
edges.shape, edges[edges.Weight >= 3].shape

((2383883, 7), (454260, 7))

## Initialize functional graph

In [21]:
graph = nx.DiGraph()

In [ ]:
# nodes_list = [[tup.Node,{"id":tup.Node_id, "taxon":tup.Node_Taxon}] for tup in nodes.itertuples()]
# print(nodes_list[0:10])
# graph.add_nodes_from(nodes_list)

In [23]:
edge_list = [(tup.Source_node, tup.Destination_node, {"weight":tup.Weight}) 
             for tup in edges.itertuples() if tup.Weight >= 3]

In [25]:
import gzip

In [31]:
with gzip.open(os.path.join(processed_network, "graphsage_test.csv.gz"), "wb") as writer:
    writer.write("source\ttarget\tweight\n".encode())
    for src,tgt,d in edge_list:
        writer.write("{}\t{}\t{}\n".format(src,tgt,d['weight']).encode())

In [ ]:
graph.add_edges_from(edge_list)

In [ ]:
nx.info(graph)

In [ ]:
nx.is_connected(graph.to_undirected())

In [ ]:
len(list(nx.connected_components(graph.to_undirected())))

In [ ]:
for i,cc in enumerate(nx.connected_components(graph.to_undirected())):
    if len(cc) > 10000:
        print(i,len(cc))

In [ ]:
largest_cc = max(nx.connected_components(graph.to_undirected()), key=len)
list(largest_cc)[0:10]

In [ ]:
subgraph = graph.subgraph(largest_cc)
nx.info(subgraph)

    'Name: \nType: DiGraph\nNumber of nodes: 146748\nNumber of edges: 514181\nAverage in degree:   3.5038\nAverage out degree:   3.5038'

In [ ]:
attrs = {"node_id":{},"node_taxon":{}}
for tup in nodes.itertuples():
    if tup.Node in subgraph.nodes():
        attrs["node_id"][tup.Node] = tup.Node_id
        attrs["node_taxon"][tup.Node] = tup.Node_Taxon

In [ ]:
list(attrs["node_id"].items())[0:10]

In [ ]:
nx.set_node_attributes(graph, attrs["node_id"], 'nid')
nx.set_node_attributes(graph, attrs["node_taxon"], 'taxon')

In [ ]:
for node,data in list(subgraph.nodes(data=True))[0:2]:
    print(node,data.keys())

### Compute shortest path

In [ ]:
short_paths = nx.shortest_path(subgraph, source="/")

In [ ]:
len(short_paths)

In [ ]:
count = 0
for key,value in short_paths.items():
    if len(value) > 15:
        count+=1
count

#### Average shortest path across all nodes

In [ ]:
# %timeit nx.average_shortest_path_length(subgraph)

#### Djikstra and A*

In [ ]:
# %timeit djik_pairs = nx.all_pairs_dijkstra_path(subgraph,weight='weight')

In [ ]:
a_star_paths = {}
for i,(src,d) in enumerate(list(subgraph.nodes(data=True))[0:100]):
    a_star_paths[src] = {}
    print(i)
    for dest,d2 in list(subgraph.nodes(data=True))[0:100]:
        if src != dest:
            path = None
            try:
                path = nx.astar_path(subgraph, src, dest, weight='weight')        
            except nx.NetworkXNoPath:
                print("no path:",src,dest)
            a_star_paths[src][dest] = path

In [ ]:
# filename = os.path.join(processed_network,"sampled_clean_nodes.csv.gz")
# import gzip
# with gzip.open(filename,"wb") as writer:
#     writer.write("Node\n".encode())
#     for node in subgraph.nodes():
#         writer.write("{}\n".format(node).encode())

### Subtrees from all pages, compute subtree depth

In [ ]:
home_edges = list(nx.dfs_edges(subgraph, source='/'))

In [ ]:
home_graph = subgraph.edge_subgraph(home_edges).copy()  

In [ ]:
len(nx.shortest_path_length(home_graph, "/"))

## Degree histogram 

In [ ]:
max_in = max([d for n, d in subgraph.in_degree()])
max_out = max([d for n, d in subgraph.out_degree()])
max_all = max([d for n, d in subgraph.degree()])
max_in,max_out,max_all

In [ ]:
subgraph.in_degree("/"),subgraph.out_degree("/")

In [ ]:
high_in = {"https://www.gov.uk"+n : d for n, d in subgraph.in_degree() if d > 500}

In [ ]:
for key,value in high_in.items():
    print(key,value)

In [ ]:
[n for n, d in subgraph.degree() if d == 9505]

In [ ]:
degree_sequence = sorted([d for n, d in subgraph.degree()], reverse=True)  # degree sequence
# print "Degree sequence", degree_sequence
degreeCount = Counter(degree_sequence)
deg, cnt = zip(*degreeCount.items())
plt.figure(figsize=(20,10))
plt.plot(deg, cnt, '.')
plt.xscale('log')
plt.yscale('log')
plt.title('Degree distribution')
txt = "For pages extracted from edge_weight >= 3 sampled functional network 28-31/01/19."
plt.figtext(0.5, 0.01, txt, wrap=True, horizontalalignment='center', fontsize=12)
plt.xlabel('K')
plt.ylabel('P_K')
plt.show()